In [4]:
# Dependencies 
import pandas as pd
import numpy as np
import os

from sqlalchemy import create_engine
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from config import dbuser, dbpasswd, dburi, dbport, dbname
Base = declarative_base()

In [5]:
# Read CSV files into a pandas DataFrame
hawaii_measurements_df = pd.read_csv("data/clean_hawaii_measurements.csv", dtype=object)
hawaii_stations_df = pd.read_csv("data/clean_hawaii_stations.csv", dtype=object)

In [6]:
# Create a Database Connection
engine = create_engine('sqlite:///hawaii.sqlite')
Base.metadata.create_all(engine)

In [7]:
from sqlalchemy.orm import Session
session = Session(bind=engine)